In [36]:
# Prepare_sdf_samples:
!python geometric_model/prepare_sdf_samples.py --input_folder /home/lucasp/thesis/TopCoW/gt_train --output_folder /home/lucasp/thesis/npz_output --num_points 15000 --latent_dim 256

Processing segmentation masks:   0%|                     | 0/10 [00:00<?, ?it/s]Bounding box: x=(71, 216), y=(95, 198), z=(53, 101)
Cropped volume shape: (146, 104, 49)
Histogram saved to surface_noisy_histogram.png
Processing segmentation masks:  10%|█▎           | 1/10 [00:00<00:03,  2.35it/s]Bounding box: x=(91, 214), y=(97, 208), z=(60, 119)
Cropped volume shape: (124, 112, 60)
Histogram saved to surface_noisy_histogram.png
Processing segmentation masks:  20%|██▌          | 2/10 [00:00<00:02,  2.92it/s]Bounding box: x=(107, 284), y=(74, 244), z=(71, 139)
Cropped volume shape: (178, 171, 69)
Histogram saved to surface_noisy_histogram.png
Processing segmentation masks:  30%|███▉         | 3/10 [00:01<00:02,  2.37it/s]Bounding box: x=(65, 223), y=(50, 203), z=(21, 83)
Cropped volume shape: (159, 154, 63)
Histogram saved to surface_noisy_histogram.png
Processing segmentation masks:  40%|█████▏       | 4/10 [00:01<00:02,  2.55it/s]Bounding box: x=(65, 224), y=(97, 201), z=(46, 100)
Crop

In [38]:
# testing with init_latent
!python geometric_model/prepare_sdf_samples.py --input_folder /home/lucasp/thesis/TopCoW/gt_train --output_folder /home/lucasp/thesis/npz_output_test --init_latents
# NOTE: Didnt change anything

Processing segmentation masks:   0%|                     | 0/10 [00:00<?, ?it/s]Bounding box: x=(71, 216), y=(95, 198), z=(53, 101)
Cropped volume shape: (146, 104, 49)
Shape: Negative voxels: 6720 (0.90%), Positive voxels: 737296 (99.10%)
Histogram saved to surface_noisy_histogram.png
Processing segmentation masks:  10%|█▎           | 1/10 [00:00<00:04,  2.11it/s]Bounding box: x=(91, 214), y=(97, 208), z=(60, 119)
Cropped volume shape: (124, 112, 60)
Shape: Negative voxels: 11319 (1.36%), Positive voxels: 821961 (98.64%)
Histogram saved to surface_noisy_histogram.png
Processing segmentation masks:  20%|██▌          | 2/10 [00:00<00:03,  2.65it/s]Bounding box: x=(107, 284), y=(74, 244), z=(71, 139)
Cropped volume shape: (178, 171, 69)
Shape: Negative voxels: 18507 (0.88%), Positive voxels: 2081715 (99.12%)
Histogram saved to surface_noisy_histogram.png
Processing segmentation masks:  30%|███▉         | 3/10 [00:01<00:03,  2.17it/s]Bounding box: x=(65, 223), y=(50, 203), z=(21, 83)
Crop

In [ ]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def load_npz_samples(npz_file):
    """
    Loads the .npz file produced by your preparation script.
    Expects the file to contain:
       - "pos": numpy array of shape [N, 3] with the voxel coordinates
       - "neg": numpy array of shape [N] with the corresponding SDF values.
    """
    data = np.load(npz_file)
    return data['pos'], data['neg']

def plot_full_sdf_scatter(coords, sdf_values):
    """
    Creates an interactive 3D scatter plot of all sample points,
    color-coded by their SDF value.
    """
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    sc = ax.scatter(coords[:, 0], coords[:, 1], coords[:, 2], c=sdf_values,
                    cmap='coolwarm', marker='o', s=5, alpha=0.8)
    plt.colorbar(sc, ax=ax, label='SDF Value')
    ax.set_title("3D Scatter Plot of SDF Samples")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    
    plt.show()  # displays the figure interactively

def plot_surface_scatter(coords, sdf_values, threshold=1.0):
    """
    Creates an interactive 3D scatter plot showing only points where 
    |SDF| < threshold (i.e. points near the boundary).
    """
    mask = np.abs(sdf_values) < threshold
    surface_coords = coords[mask]
    surface_sdf = sdf_values[mask]
    
    if surface_coords.size == 0:
        print("No points found near the surface with the given threshold.")
        return
    
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    sc = ax.scatter(surface_coords[:, 0], surface_coords[:, 1], surface_coords[:, 2],
                    c=surface_sdf, cmap='coolwarm', marker='o', s=10)
    plt.colorbar(sc, ax=ax, label=f'SDF Value (|SDF| < {threshold})')
    ax.set_title(f"3D Scatter Plot of Surface Samples (|SDF| < {threshold})")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    
    plt.show()
    
def check_near_surface_points(coords, sdf_values, lower_bound=-0.1, upper_bound=0.1):
    """
    Checks for points with SDF values in the range [lower_bound, upper_bound].
    If found, it prints how many such points exist and displays one example point.
    """
    # Create a mask for SDF values in the specified interval.
    mask = (sdf_values >= lower_bound) & (sdf_values <= upper_bound)
    count = np.sum(mask)
    print(f"Number of points with SDF in the range [{lower_bound}, {upper_bound}]: {count}")
    if count > 0:
        idx = np.where(mask)[0][0]
        print(f"Example point:\n  Coordinates: {coords[idx]}\n  SDF Value: {sdf_values[idx]}")
    else:
        print("No points found in the specified SDF range.")

if __name__ == "__main__":
    npz_path = "/home/lucasp/thesis/npz_output/topcow_ct_001.npz"
    
    coords, sdf_values = load_npz_samples(npz_path)
    
    plot_full_sdf_scatter(coords, sdf_values)
    plot_surface_scatter(coords, sdf_values, threshold=0.105)
    
    # check_near_surface_points(coords, sdf_values, lower_bound=-.11, upper_bound=.11)

In [37]:
# Training:
!python /home/lucasp/thesis/main.py --npz_folder /home/lucasp/thesis/npz_output --num_epochs 2500 --do_code_regularization

Found 10 shapes in dataset.
/home/lucasp/miniconda3/envs/nnunet/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
[Epoch 1/2500] Loss: 0.411413 | Time: 0.39s | Mean latent norm: 0.999365
New best model saved at epoch 1 with loss 0.411413
[Epoch 2/2500] Loss: 0.386310 | Time: 0.19s | Mean latent norm: 0.999404
New best model saved at epoch 2 with loss 0.386310
[Epoch 3/2500] Loss: 0.386970 | Time: 0.19s | Mean latent norm: 0.999337
[Epoch 4/2500] Loss: 0.386068 | Time: 0.19s | Mean latent norm: 0.999229
New best model saved at epoch 4 with loss 0.386068
[Epoch 5/2500] Loss: 0.384780 | Time: 0.19s | Mean latent norm: 0.999122
New best model saved at epoch 5 with loss 0.384780
[Epoch 6/2500] Loss: 0.383934 | Time: 0.19s | Mean latent norm: 0.999040
New best model saved at epoch 6 with loss 0.383934
[Epoch 7/2500] Loss: 0.3

In [27]:
# Testing on sphere:
!python /home/lucasp/thesis/main.py --npz_folder /home/lucasp/thesis/sphere_test --num_epochs 500 --do_code_regularization

Found 1 shapes in dataset.
/home/lucasp/miniconda3/envs/nnunet/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
[Epoch 1/500] Loss: 0.358405 | Time: 0.22s | Mean latent norm: 0.971923
Average difference with latent perturbation: 0.007071135565638542
New best model saved at epoch 1 with loss 0.358405
[Epoch 2/500] Loss: 0.357094 | Time: 0.03s | Mean latent norm: 0.971923
New best model saved at epoch 2 with loss 0.357094
[Epoch 3/500] Loss: 0.355518 | Time: 0.03s | Mean latent norm: 0.971931
New best model saved at epoch 3 with loss 0.355518
[Epoch 4/500] Loss: 0.354092 | Time: 0.02s | Mean latent norm: 0.971950
New best model saved at epoch 4 with loss 0.354092
[Epoch 5/500] Loss: 0.352720 | Time: 0.02s | Mean latent norm: 0.971976
New best model saved at epoch 5 with loss 0.352720
[Epoch 6/500] Loss: 0.351168 | Time: 

Changing GT image for reconstruction/visualization, it only seems to be moving the reconstructed shape around without actually changing its shape which means that it doesn't seem to be using the latent vectors appropriately or there is an issue with how they're being integrated in the model...

In [13]:
# Visualization:
!python main.py --visualize_only --segmentation_file /home/lucasp/thesis/TopCoW/gt_train/topcow_ct_001.nii.gz

/home/lucasp/thesis/main.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("auto_decoder_model.pth", map_location="cuda")
/home/lucasp/miniconda3/

In [38]:
# To reconstruct mesh:
!python geometric_model/reconstruct_mesh.py --checkpoint auto_decoder.pth --shape_idx 5 --output gt_001_2500_epochs_15k_samples.ply --resolution 512
# !python geometric_model/reconstruct_mesh.py --checkpoint auto_decoder.pth --shape_idx 5 --output gt_001_2500_epochs_25k_samples_384.ply --resolution 512 --hidden_dims "384,384,384,384,384,384,384,384"

/home/lucasp/thesis/geometric_model/reconstruct_mesh.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(checkpoint_path, map_location=device)
/home/lucas

In [ ]:
# TODO: To run inference on trained model:
!python geometric_model/reconstruct_mesh.py --checkpoint auto_decoder_2500_epochs.pth --segmentation TopCoW/2d_base_without_mirroring/topcow_ct_001.nii.gz --output topcow_ct_001_recon.ply --resolution 512 --opt_num_iters 5000

/home/lucasp/thesis/geometric_model/reconstruct_mesh.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(checkpoint_path, map_location=device)
/home/lucas